In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
#plt.style.use('bmh')
plt.style.use("seaborn-colorblind")
import sys, os
import pynoddy.history
import pynoddy.output
import pynoddy.experiment
import pymc
import theano.tensor as T
import GeoMig
import importlib
#import geogrid
reload(GeoMig)
import pickle
import importlib
#importlib.reload(GeoMig)
import numpy as np
import pandas as pn
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
from skimage import measure
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
np.set_printoptions(precision = 6, linewidth= 130, suppress =  True)
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib notebook
import pandas as pd
import pickle
sys.path.append("..")
import PosteriorAnalysis as pa

##### Load Interface Priors

# GeMpy Model

In [ ]:
nx = 200
nz = 100

GM = GeoMig.Interpolator(0,nx,
                         0,10,
                         0,nz,
                         u_grade=3)

# set grid resolution
GM.set_resolutions(nx,4,nz)
GM.create_regular_grid_3D()

# compile
GM.theano_compilation_3D()

#
_interfaces_col_names = ["X", "Y", "Z", "formation"]
interfaces = pn.DataFrame(columns=_interfaces_col_names)
for row in mean_prior_interfaces:
    interfaces.loc[len(interfaces)] = [row[0], row[1], row[2], row[3]]
    
GM.load_data_pd("interfaces", interfaces)

In [ ]:
def calc_fucking_foliations(interface_df):
    test_sorted = interface_df.sort_values("formation")
    _foliation_col_names = ["X", "Y", "Z", "azimuth", "dip", "polarity", "formation"]
    foliations = pn.DataFrame(columns=_foliation_col_names)

    for i,row in enumerate(test_sorted.values):
        #print i,row
        if row[3] == "Layer3" or row[3] == "Layer5":
            if i != len(test_sorted.values)-1:
                if row[3] == test_sorted.values[i+1][3]:
                    dx = row[0] - test_sorted.values[i+1][0]
                    dz = row[2] - test_sorted.values[i+1][2]

                    dip = np.rad2deg(np.arctan(dz/float(dx)))
                    #print dip
                    foliations.loc[len(foliations)] = [np.abs((row[0] + test_sorted.values[i+1][0])/2),
                                                       0,
                                                         np.abs((row[2] + test_sorted.values[i+1][2]) / 2),
                                                         -90,
                                                         dip,
                                                         1.0,
                                                         row[3]]
    return foliations

In [ ]:
GM.load_data_pd("foliations", calc_fucking_foliations(GM.Interfaces))
print GM.Foliations
print GM.Interfaces

initial = []
for layer in GM.formations:
    if layer != "Layer5":
        initial.append(layer)
GM.set_series({"Initial": tuple(initial), "Unconformity": "Layer5"})

GM.block.set_value(np.zeros_like(GM.grid[:,0]))
GM.compute_block_model([0,1], verbose = 0)
plot_block =  GM.block.get_value().reshape(200,4,100)

%matplotlib inline
plt.imshow(plot_block[:,2,:].T,origin="lower")
plt.scatter(GM.Interfaces["X"],GM.Interfaces["Z"])
plt.scatter(GM.Foliations["X"],GM.Foliations["Z"], color="green")
#plt.savefig("PGM2_5_gempy_mean_prior_section.pdf")

# Extract Likelihood From Noddy

In [ ]:
noddy_db = # .hdf5 file of noddy experiment from which to extract the likelihood functions

In [ ]:
reload(pa)
nl = pa.PosteriorAnalysis(noddy_db, "pynoddy")
like_layer_heights = nl.kde_dict(nl.extract_layer_height_dict([5, 50, 90, 150, 180], drop=4))

for p in like_layer_heights.keys():
    for l in like_layer_heights[p].keys():
        print p,l,like_layer_heights[p][l]

### Saving as Pickle

In [ ]:
pickle.dump(like_layer_heights, open("like_layer_heights.p", "wb" ) )

#### Load Pickle

In [ ]:
like_layer_heights = pickle.load(open("like_layer_heights.p", "rb"))
like_layer_heights

# GeMpy PYMC

In [ ]:
@pymc.deterministic
def gempy_model(value=0, interf=interfaces, priors=mean_prior_interfaces, verbose=0, nx=nx, nz=nz):
    
    # set prior interface values
    interf["Z"] = priors[:,4]
    GM.load_data_pd("interfaces", interf)
    
    if verbose == 1:
        print GM.Interfaces
    
    GM.load_data_pd("foliations", calc_fucking_foliations(GM.Interfaces))
    
    if verbose == 1:
        print GM.Foliations
    
    # load layers into series
    initial = []
    for layer in GM.formations:
        if layer != "Layer5":
            initial.append(layer)
    GM.set_series({"Initial": tuple(initial), "Unconformity": "Layer5"})
    
    if verbose==1:
        print GM.series
    
    # gempy magic calculation
    GM.block.set_value(np.zeros_like(GM.grid[:,0]))
    GM.compute_block_model([0,1],verbose = 0)
    plot_block =  GM.block.get_value().reshape(200,4,100)
    
    # return gempy block model
    if verbose > 0:
        plt.imshow(plot_block[:,2,:].T, origin="lower")
        plt.plot(GM.Interfaces["X"],GM.Interfaces["Z"],"o")
        plt.plot(GM.Foliations["X"],GM.Foliations["Z"],"x", color="red")
        plt.xlim(0,200)
        plt.ylim(0,100)
    return plot_block[:,2,:]

### Likelihood Functions

In [ ]:
# layer 1
@pymc.stochastic
def like_xpos0_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 1.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 1.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
#@pymc.stochastic
#def like_xpos2_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
#    x_pos=np.sort(like_layer_heights.keys())
#    if 1.0 in gempy_model[x_pos[2],:]:
#        model_height = np.where(gempy_model[x_pos[2],:]==1.0)[0][0]
#        return np.log(like_layer_heights[x_pos[2]][1].evaluate(model_height))[0]
#    else:
#        return np.log(0.001)
    
@pymc.stochastic
def like_xpos3_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 1.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos4_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 1.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# layer 2
@pymc.stochastic
def like_xpos0_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 2.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 2.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos2_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 2.0 in gempy_model[x_pos[2],:]:
        model_height = np.where(gempy_model[x_pos[2],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[2]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos3_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 2.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos4_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 2.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# layer 3
@pymc.stochastic
def like_xpos0_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 3.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 3.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos2_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 3.0 in gempy_model[x_pos[2],:]:
        model_height = np.where(gempy_model[x_pos[2],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[2]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos3_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 3.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos4_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 3.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)

# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# layer 5
@pymc.stochastic
def like_xpos0_layer_5(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 4.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==4.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][5].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_5(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 4.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==4.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][5].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos2_layer_5(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 4.0 in gempy_model[x_pos[2],:]:
        model_height = np.where(gempy_model[x_pos[2],:]==4.0)[0][0]
        return np.log(like_layer_heights[x_pos[2]][5].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos3_layer_5(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 4.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==4.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][5].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos4_layer_5(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=np.sort(like_layer_heights.keys())
    if 4.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==4.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][5].evaluate(model_height))[0]
    else:
        return np.log(0.001)

In [ ]:
like_list = [like_xpos0_layer_1,like_xpos1_layer_1,#like_xpos2_layer_1,
             like_xpos3_layer_1,like_xpos4_layer_1,
             like_xpos0_layer_2,like_xpos1_layer_2,like_xpos2_layer_2,like_xpos3_layer_2,like_xpos4_layer_2,
             like_xpos0_layer_3,like_xpos1_layer_3,like_xpos2_layer_3,like_xpos3_layer_3,like_xpos4_layer_3,
             like_xpos0_layer_5,like_xpos1_layer_5,like_xpos2_layer_5,like_xpos3_layer_5,like_xpos4_layer_5]

In [ ]:
params = [mean_prior_interfaces[i,4] for i in range(len(mean_prior_interfaces[:,4]))]
params.append(gempy_model)
for entry in like_list:
    params.append(entry)
model = pymc.Model(params)

In [ ]:
%matplotlib inline

In [ ]:
iterations = 

In [ ]:
RUN = pymc.MCMC(model, db="hdf5", name="")
RUN.sample(iter=iterations)

In [ ]:
plt.imshow(RUN.trace("gempy_model")[2500].T, origin="lower")